Create "wild-type" reads (no SNVs).

In [17]:
%%bash

ref="test_input/e_coli/NC_008253_1K.fna"
log_dir="test_input/sim_wt/sim_logs"
output_file="test_input/sim_wt/sim_wt.sorted.bam"

fq_file="$(mktemp)"
sam_file="$(mktemp)"
bam_file="$(mktemp)"

mkdir -p "$(dirname $output_file)"
mkdir -p "$log_dir"

set -euxo pipefail

# Generate 1,000 reads, with a 0% rate of mutations and 0% error rate.
# Set seed to a constant for reproducability.
# wgsim outputs paired-end reads; we send second ends to /dev/null to get single-end reads.
wgsim -N 1000 -r 0 -S 8 -e 0 "$ref" "$fq_file" /dev/null 2>&1 >"$log_dir"/wgsim.log
bwa mem -M -t 8 -p "$ref" "$fq_file" >>"$sam_file" 2>"$log_dir"/bwa.log
samtools view -b -S -o "$bam_file" "$sam_file"
samtools sort "$bam_file" -o "$output_file"
samtools index "$output_file"

# Clean up intermediate files
rm "$fq_file"
rm "$sam_file"
rm "$bam_file"

[wgsim] seed = 8
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 1000


+ wgsim -N 1000 -r 0 -S 8 -e 0 test_input/e_coli/NC_008253_1K.fna /tmp/tmp.Vp2SDxNEMq /dev/null
+ bwa mem -M -t 8 -p test_input/e_coli/NC_008253_1K.fna /tmp/tmp.Vp2SDxNEMq
+ samtools view -b -S -o /tmp/tmp.4ZljxTN0c2 /tmp/tmp.j8uu7qTeaU
+ samtools sort /tmp/tmp.4ZljxTN0c2 -o test_input/sim_wt/sim_wt.sorted.bam
+ samtools index test_input/sim_wt/sim_wt.sorted.bam
+ rm /tmp/tmp.Vp2SDxNEMq
+ rm /tmp/tmp.j8uu7qTeaU
+ rm /tmp/tmp.4ZljxTN0c2


Produce simulated mutated reads. We just need three types of mutated BAMs to construct all simulated allelic asymmetries:
* Homozygous ref (already created above).
* Heterozygous (50% ref, 50% alt)
* Homozygous alt

In [21]:
%%bash

ref="test_input/e_coli/NC_008253_1K.fna"
picard_jar="/seq/software/picard/current/bin/picard.jar"
sim_wt_bam="test_input/sim_wt/sim_wt.sorted.bam"
het_output="test_input/sim_het/sim_het.sorted.bam"
hom_output="test_input/sim_hom/sim_hom.sorted.bam"
het_logs="test_input/sim_het/logs"
hom_logs="test_input/sim_het/logs"

chrom="gi|110640213|ref|NC_008253.1|"
snv_base="C"
snv_pos="200"
het_vaf="0.5"
hom_vaf="1"

mkdir -p "$(dirname $het_output)"
mkdir -p "$(dirname $hom_output)"
mkdir -p "$het_logs"
mkdir -p "$hom_logs"
het_bam_file="$(mktemp)"
hom_bam_file="$(mktemp)"

set -euxo pipefail

# Make heterozygous BAM
spikein_file="$(mktemp)"
echo "$chrom   $snv_pos     $snv_pos     $het_vaf     $snv_base" >"$spikein_file"

bamsurgeon addsnv.py \
    --single \
    --picardjar "$picard_jar" \
    --aligner mem \
    -v "$spikein_file" \
    -f "$sim_wt_bam" \
    -r "$ref" \
    -o "$het_bam_file" \
    --tmpdir "$het_logs" \
    2>&1 >"$het_logs/bamsurgeon.log"

samtools sort "$het_bam_file" -o "$het_output"
samtools index "$het_output"
mv "addsnv_logs_tmp."* "$het_logs"

# Make homozygous BAM
echo "$chrom   $snv_pos     $snv_pos     $hom_vaf     $snv_base" >"$spikein_file"

bamsurgeon addsnv.py \
    --single \
    --picardjar "$picard_jar" \
    --aligner mem \
    -v "$spikein_file" \
    -f "$sim_wt_bam" \
    -r "$ref" \
    -o "$hom_bam_file" \
    --tmpdir "$hom_logs" \
    2>&1 >"$hom_logs/bamsurgeon.log"

samtools sort "$hom_bam_file" -o "$hom_output"
samtools index "$hom_output"
mv "addsnv_logs_tmp."* "$hom_logs"

rm "$het_bam_file"
rm "$hom_bam_file"
rm "$spikein_file"

[Fri Apr 14 15:56:40 EDT 2017] picard.sam.SamToFastq INPUT=test_input/sim_het/logs/haplo_gi|110640213|ref|NC_008253.1|_200_200.tmpbam.d208127a-baba-47dc-9ce9-cfdfd398239c.bam FASTQ=test_input/sim_het/logs/haplo_gi|110640213|ref|NC_008253.1|_200_200.tmpbam.d208127a-baba-47dc-9ce9-cfdfd398239c.fastq INCLUDE_NON_PRIMARY_ALIGNMENTS=false VALIDATION_STRINGENCY=SILENT    OUTPUT_PER_RG=false RG_TAG=PU RE_REVERSE=true INTERLEAVE=false INCLUDE_NON_PF_READS=false CLIPPING_MIN_LENGTH=0 READ1_TRIM=0 READ2_TRIM=0 VERBOSITY=INFO QUIET=false COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json
[Fri Apr 14 15:56:40 EDT 2017] Executing as moorena@ccpm.broadinstitute.org on Linux 2.6.32-642.13.1.el6.x86_64 amd64; OpenJDK 64-Bit Server VM 1.8.0_92-b15; Picard version: 2.9.0-SNAPSHOT
[Fri Apr 14 15:56:41 EDT 2017] picard.sam.SamToFastq done. Elapsed time: 0.00 minutes.
Runtime.totalMemory()=1012924416
[M::bwa_idx_load_from_disk] re

++ mktemp
+ spikein_file=/tmp/tmp.fsSYJNdaVg
+ echo 'gi|110640213|ref|NC_008253.1|   200     200     0.5     C'
+ bamsurgeon addsnv.py --single --picardjar /seq/software/picard/current/bin/picard.jar --aligner mem -v /tmp/tmp.fsSYJNdaVg -f test_input/sim_wt/sim_wt.sorted.bam -r test_input/e_coli/NC_008253_1K.fna -o /tmp/tmp.cPgguTAcOG --tmpdir test_input/sim_het/logs
+ samtools sort /tmp/tmp.cPgguTAcOG -o test_input/sim_het/sim_het.sorted.bam
+ samtools index test_input/sim_het/sim_het.sorted.bam
+ mv addsnv_logs_tmp.cPgguTAcOG test_input/sim_het/logs
+ echo 'gi|110640213|ref|NC_008253.1|   200     200     1     C'
+ bamsurgeon addsnv.py --single --picardjar /seq/software/picard/current/bin/picard.jar --aligner mem -v /tmp/tmp.fsSYJNdaVg -f test_input/sim_wt/sim_wt.sorted.bam -r test_input/e_coli/NC_008253_1K.fna -o /tmp/tmp.GsC4sRB1KX --tmpdir test_input/sim_het/logs
+ samtools sort /tmp/tmp.GsC4sRB1KX -o test_input/sim_hom/sim_hom.sorted.bam
+ samtools index test_input/sim_hom/sim_ho